# This is a full pipeline with keypoints prediction using VAE in reconstruction mode for VoxCeleb dataset

# Import functions

In [30]:
import os   
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
from Training_Prediction.FOMM.Source_Model.logger import Logger, Visualizer
import numpy as np
import imageio
from Training_Prediction.FOMM.Source_Model.sync_batchnorm import DataParallelWithCallback
from Training_Prediction.PREDICTOR.VAE import VAE_origin
from Training_Prediction.FOMM.Source_Model.augmentation import SelectRandomFrames, SelectFirstFrames_two, VideoToTensor
from tqdm import trange
from torch.utils.data import DataLoader, Dataset
from Training_Prediction.FOMM.Source_Model.frames_dataset import FramesDataset
import tensorflow.compat.v1 as tf
import pickle
from Training_Prediction.PREDICTOR.Source_Model.prediction_toplevel import KPDataset,get_data_from_dataloader_60
import gc
import pickle
import yaml
from Training_Prediction.FOMM.Source_Model.modules.generator import OcclusionAwareGenerator,calculate_frechet_distance,compute_fvd
from Training_Prediction.FOMM.Source_Model.modules.keypoint_detector import KPDetector
from Training_Prediction.FOMM.Source_Model.logger import Logger, Visualizer, Visualizer_slow
from torch import nn
import tensorflow.compat.v1 as tf
from torch.autograd import Variable
import random
import matplotlib.pyplot as plt
import vmaf
import subprocess
import imageio
import json
import os, sys
from PIL import Image
import shutil

import os, sys
os.environ["CUDA_VISIBLE_DEVICES"]='0'

In [31]:
def save_frames_as_png(frames, png_dir):
    for idx, frame in enumerate(frames):
        frame_path = os.path.join(png_dir, f'video_frame_{idx:04d}.png')
        imageio.imsave(frame_path, (255 * frame).astype(np.uint8))

In [32]:
def convert_png_to_mp4(png_dir,mp4_dir,video_number):
    if not png_dir.endswith('/'):
        png_dir += '/'
    if not mp4_dir.endswith('/'):
        mp4_dir += '/'
    
    input_pattern = f"{png_dir}video_frame_%04d.png"
    output_file = f"{mp4_dir}video_{video_number:02d}.mp4"
    command = (f'ffmpeg -y -r 60 -i {png_dir}/video_frame_%04d.png -pix_fmt yuv420p -profile:v high -level:v 4.1 -crf:v 20 -movflags +faststart {mp4_dir}/video_{video_number:02d}.mp4')
    os.system(command)

In [33]:
def convert_png_to_y4m(png_dir, y4m_dir, video_number):
    if not png_dir.endswith('/'):
        png_dir += '/'
    if not y4m_dir.endswith('/'):
        y4m_dir += '/'
    
    input_pattern = f"{png_dir}video_frame_%04d.png"
    output_file = f"{y4m_dir}video_{video_number:02d}.y4m"
    
    command = (f'ffmpeg -y -i {input_pattern} -r 24 -pix_fmt yuv444p {output_file}')    
    os.system(command)

In [34]:
def run_vmaf(driving_vid, generated_vid, output_path):
    if driving_vid[-3:] != 'y4m':
         raise Exception('Video must be in y4m format.')
    if generated_vid[-3:] != 'y4m':
         raise Exception('Video must be in y4m format.')
    if output_path[-4:] != 'json':
        raise Exception('Output file must be in json format')
    
    command = (f'vmaf --reference {driving_vid} --distorted {generated_vid} --model version=vmaf_v0.6.1 --output {output_path} --json')
    os.system(command)

# Define VAE model

In [35]:
lag_time = 12


model = VAE_origin(60, lag_time=lag_time,
          hidden_size=256, hidden_layer_depth=3,
          batch_size=250, n_epochs=100, cuda=True, 
          sliding_window=True, dropout_rate=0.3,
          learning_rate=1E-3, scale=1E-3, autocorr=False, encoder_size=20)

# After VAE is trained, load saved model

In [36]:
model.load_state_dict(torch.load('Checkpoints/VAE_3883videos_vox_12-12.pth')) # 12-12 frames
# model.load_state_dict(torch.load('Checkpoints/VAE_3883videos_vox_6-6.pth')) # 6-6 frames

# Set the model to evaluation mode (important if using dropout or batch normalization)
model.eval()

VDE(input_size=60, encoder_size=20, n_epochs=100,
    batch_size=250, lag_time=12, sliding_window=True,
    autocorr=False, cuda=True)

# Import keypoints of 44 VoxCeleb test videos

In [37]:
with open("kp_test_44_vox.pkl", "rb") as f:
    kp_time_series = pickle.load(f)
len(kp_time_series)

44

# Convert list of keypoints to dictionary

In [38]:
for video_idx in range(len(kp_time_series)):
    kp_time_series[video_idx] = kp_time_series[video_idx]['kp']

kp_dict_init = []
for video_idx in range(len(kp_time_series)): # 
    init_mean = []
    init_jacobian = []
    for frame_idx in range(len(kp_time_series[video_idx])):
        kp_mean = kp_time_series[video_idx][frame_idx]['value'].reshape(1,10,2)
        kp_mean = torch.tensor(kp_mean)
        kp_jacobian = kp_time_series[video_idx][frame_idx]['jacobian'].reshape(1,10,2,2)
        kp_jacobian = torch.tensor(kp_jacobian)

        init_mean.append(kp_mean)
        init_jacobian.append(kp_jacobian)

    init_mean = torch.cat(init_mean)
    init_jacobian = torch.cat(init_jacobian)

    init_mean = torch.reshape(init_mean,(1,init_mean.shape[0],init_mean.shape[1],init_mean.shape[2]))
    init_jacobian = torch.reshape(init_jacobian,(1,init_jacobian.shape[0],10,2,2))

    if torch.cuda.is_available():
        # add tensor to cuda
        init_mean = init_mean.to('cuda:0')
        init_jacobian = init_jacobian.to('cuda:0')

    kp_dict_both = {"value":init_mean,"jacobian":init_jacobian}
    kp_dict_init.append(kp_dict_both)

# Apply min-max std to keypoints and convert to batches


In [39]:
kp_list_test = []
for video_idx in range(len(kp_dict_init)):
    kp_one_video = torch.cat((kp_dict_init[video_idx]['value'], kp_dict_init[video_idx]['jacobian'].reshape(1,-1,10,4)),dim=-1).reshape(-1,60)
    kp_one_video_array = np.array(kp_one_video.cpu())
    kp_list_test.append(kp_one_video_array)
    
#####  min-max std to 60 dimensions of selected one video ######
kp_list_test_std = []
min_list = []
range_list = []
for video_idx in range(len(kp_list_test)):
    data = kp_list_test[video_idx]
    data_length = len(kp_list_test[video_idx])
    step_interval = 12 # choose between 12 frames or 24 frames 
    min_required_steps = 2*step_interval
    selected_data = []
    for i in range(0, data_length - min_required_steps+1, 2 * step_interval):
        selected_data.extend(data[i:i + step_interval])
    min_values = np.min(selected_data,axis=0) # 60 mins of one selected video in the loop
    max_values = np.max(selected_data,axis=0) # 60 maxs of one selected video in the loop 
    range_values = max_values - min_values 
    kp_one_video_std = (kp_list_test[video_idx] - min_values) / range_values
    kp_list_test_std.append(kp_one_video_std)
    min_list.append(min_values)
    range_list.append(range_values)

trajs = kp_list_test_std
print(len(trajs))
print(trajs[0].shape)

44
(118, 60)


In [40]:
######### convert into batches:
frames = min_required_steps
input_frames = int(frames / 2)
data_batch_test = []
for t,x in enumerate(kp_list_test_std):
    if x.shape[0] >= frames:
        num_full_batches = x.shape[0] // frames
        for arr in np.array_split(x[:num_full_batches * frames], num_full_batches):
            data_batch_test.append(arr)
print(f'test dataset batches:', len(data_batch_test))
print(data_batch_test[0].shape)

test dataset batches: 529
(24, 60)


In [41]:
###### test dataset:

test_data_reshape = np.array(data_batch_test).reshape(-1,frames,60)
test_data_reshape.shape

(529, 24, 60)

# Predict keypoints using trained model:

In [42]:
# test dataset
validation_data = test_data_reshape

# evaluate model:
validation_input = torch.tensor(validation_data[:,:input_frames], dtype = torch.float32) # input: [24,10,17]
kp_gt = torch.tensor(validation_data[:,input_frames:], dtype = torch.float32) # gtoundtruth: [24,10,17]

# Move the model to the desired device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# Check the device of your input tensor (x)
validation_input = torch.tensor(validation_input).to(device)

pred_encoded = model.encoder(validation_input)
pred_lmbd = model.lmbd(pred_encoded)
pred = model.decoder(pred_lmbd)

print(kp_gt.shape)
print(pred.shape)

torch.Size([529, 12, 60])
torch.Size([529, 12, 60])


/tmp/ipykernel_6337/207256912.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  validation_input = torch.tensor(validation_input).to(device)


# Generate unstd keypoints:

In [43]:
# save num_batches for each video:
num_batch_video = []
num_full_batches_all = 0
for t,x in enumerate(kp_list_test_std):
    if x.shape[0] > frames:
        num_full_batches = x.shape[0] // frames
        num_full_batches_all += num_full_batches
        num_batch_video.append(num_full_batches)
print(f'number of batches of each video:', len(num_batch_video))

number of batches of each video: 44


In [44]:
# first half of frames: groundtruth; last half of frames: predicted
test_gt_pred = np.concatenate((test_data_reshape[:,:input_frames], pred.detach().cpu()), axis = 1)

test_gt_pred.shape

(529, 24, 60)

In [45]:
#for no prediction
test_video_unstd_list = kp_time_series

In [46]:
# unstd for each video -- for prediction
test_video_unstd_list = []
for video_idx in range(len(num_batch_video)):
    test_video = test_gt_pred[sum(num_batch_video[:video_idx]):sum(num_batch_video[:video_idx+1])]
    test_video_unstd = test_video * range_list[video_idx] + min_list[video_idx]
    test_video_unstd_list.append(test_video_unstd) # unstd video keypoints

# Optical flow and generator

In [47]:
####### call the config functions and inference dataloader #########
config="config/abs-vox.yml"

# Test dataset
with open(config) as f:
    config = yaml.safe_load(f)
dataset = FramesDataset(is_train=(False), **config['dataset_params'],mode="RNN") # test

print(len(dataset))
dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=1)

### call the functions        
generator = OcclusionAwareGenerator(**config['model_params']['generator_params'],
                                        **config['model_params']['common_params'])
kp_detector = KPDetector(**config['model_params']['kp_detector_params'],
                            **config['model_params']['common_params'])

log_dir="./log/test-reconstruction-vox"
checkpoint="./Training_Prediction/FOMM/Trained_Models/vox-cpk.pth.tar"

if checkpoint is not None:
    Logger.load_cpk(checkpoint, generator=generator, kp_detector=kp_detector)
else:
    raise AttributeError("Checkpoint should be specified for mode='reconstruction'.")
    
def save_obj(obj, name ):
    with open('./'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('./' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

pred_png_dir = os.path.join(log_dir, 'prediction/prediction_png')
pred_mp4_dir = os.path.join(log_dir, 'prediction/prediction_mp4')
pred_y4m_dir = os.path.join(log_dir, 'prediction/prediction_y4m')
driving_png_dir = os.path.join(log_dir, 'prediction/driving_png')
driving_mp4_dir = os.path.join(log_dir, 'prediction/driving_mp4')
driving_y4m_dir = os.path.join(log_dir, 'prediction/driving_y4m')
vae_vmaf_output = os.path.join(log_dir, 'prediction/vmaf_json_output_vae')

log_dir = os.path.join(log_dir, 'prediction')

if not os.path.exists(log_dir):
    os.makedirs(log_dir)

if not os.path.exists(pred_png_dir):
    os.makedirs(pred_png_dir)
if not os.path.exists(pred_mp4_dir):
    os.makedirs(pred_mp4_dir)
if not os.path.exists(pred_y4m_dir):
    os.makedirs(pred_y4m_dir)

if not os.path.exists(driving_png_dir):
    os.makedirs(driving_png_dir)
if not os.path.exists(driving_mp4_dir):
    os.makedirs(driving_mp4_dir)
if not os.path.exists(driving_y4m_dir):
    os.makedirs(driving_y4m_dir)

if not os.path.exists(vae_vmaf_output):
    os.makedirs(vae_vmaf_output)

if torch.cuda.is_available():
    generator = DataParallelWithCallback(generator)
    kp_detector = DataParallelWithCallback(kp_detector)

generator.eval()
kp_detector.eval()

prediction_params = config['prediction_params']

num_epochs = prediction_params['num_epochs']
lr = prediction_params['lr']
bs = prediction_params['batch_size']
num_frames = prediction_params['num_frames']
loss_list_total = []
fvd_list_total = []

Use predefined train-test split.
using videos from test directory
['id10280#NXjT3732Ekg#001093#001192.mp4', 'id10281#NHARUN9OhSo#000605#000886.mp4', 'id10281#NHARUN9OhSo#001059#001210.mp4', 'id10281#NHARUN9OhSo#002098#002175.mp4', 'id10281#NHARUN9OhSo#002209#002570.mp4', 'id10281#NHARUN9OhSo#006609#006906.mp4', 'id10281#NHARUN9OhSo#006912#007284.mp4', 'id10281#NHARUN9OhSo#007425#007663.mp4', 'id10282#IDA_ElNHLn4#000674#000852.mp4', 'id10282#IDA_ElNHLn4#001226#001390.mp4', 'id10283#N69Hp2DGMLk#000519#000619.mp4', 'id10283#N69Hp2DGMLk#000721#000842.mp4', 'id10283#N69Hp2DGMLk#000893#001589.mp4', 'id10283#N69Hp2DGMLk#004133#005157.mp4', 'id10283#N69Hp2DGMLk#005157#005316.mp4', 'id10283#N69Hp2DGMLk#005931#006184.mp4', 'id10283#N69Hp2DGMLk#006184#006353.mp4', 'id10283#N69Hp2DGMLk#006405#006583.mp4', 'id10283#N69Hp2DGMLk#006600#007118.mp4', 'id10283#N69Hp2DGMLk#007129#007281.mp4', 'id10283#r9-0pljhZqs#002414#002769.mp4', 'id10283#r9-0pljhZqs#003725#003847.mp4', 'id10283#r9-0pljhZqs#004062#004

In [52]:
#########  FOMM+VAE ########
mse_list = []
for it, x in tqdm(enumerate(dataloader)):
        if config['reconstruction_params']['num_videos'] is not None:
            if it > config['reconstruction_params']['num_videos']:
                break
        if it>1:
            break
    
        # Clear the PNG directories for each iteration of the outer loop
        shutil.rmtree(pred_png_dir, ignore_errors=True)
        os.makedirs(pred_png_dir)
    
        shutil.rmtree(driving_png_dir, ignore_errors=True)
        os.makedirs(driving_png_dir)
    
        with torch.no_grad():
            predictions = []
            driving_frames = []
            
            ######## keypoints ########
            kp_driving_video = test_video_unstd_list[it].reshape(-1,10,6)
            kp_driving_video = torch.tensor(kp_driving_video)
            kp_source = {"value":kp_driving_video[0,:,:2].reshape(1,10,2),"jacobian":kp_driving_video[0,:,2:].reshape(1,10,2,2)} # kp of the ith frame      
        mse_list_frames = []
        ##### Start generator
        for i in range(((x['video'].shape[2])//frames)*frames): # cut the last <24 frames
            source = x['video'][:, :, 0]
            driving = x['video'][:, :, i]
            kp_driving = {"value":kp_driving_video[i,:,:2],"jacobian":kp_driving_video[i,:,2:]} # kp of the ith frame
            kp_driving['value'] = kp_driving['value'].reshape(1,10,2)
            kp_driving['jacobian'] = kp_driving['jacobian'].reshape(1,10,2,2)
            out = generator(source, kp_source=kp_source, kp_driving=kp_driving)
            out['kp_source'] = kp_source
            out['kp_driving'] = kp_driving
            del out['sparse_deformed']
            driving_frames.append(np.transpose(driving.data.cpu().numpy(),[0, 2, 3, 1])[0])
            predictions.append(np.transpose(out['prediction'].data.cpu().numpy(), [0, 2, 3, 1])[0])
            mse_list_frames.append(np.mean(np.square(driving_frames[i] - predictions[i])))
#        predictions = np.concatenate(predictions, axis=1)

        mse_list.append(np.mean(mse_list_frames))

        #Save frames of predicted and driving videos as png's
        save_frames_as_png(predictions, pred_png_dir)
        save_frames_as_png(driving_frames,driving_png_dir)

        #Convert frames into a y4m-formatted video
#        convert_png_to_y4m(pred_png_dir, pred_y4m_dir, it)
#        convert_png_to_y4m(driving_png_dir, driving_y4m_dir, it)
        convert_png_to_mp4(pred_png_dir,pred_mp4_dir,it)
        convert_png_to_mp4(driving_png_dir,driving_mp4_dir,it)

        #Run VMAF on y4m-format videos
#        run_vmaf(driving_y4m_dir+f'/video_{it:02d}.y4m',pred_y4m_dir+f'/video_{it:02d}.y4m',vae_vmaf_output+f'/vmaf_{it:02d}.json')


0it [00:00, ?it/s]ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --en